# Tutorial for navigating rtmod python package
### *Jason Cory Brunson*
### *Igor Sokolov*
### September 2022

## Overview
**rtmod** is a Python repository for a collection of software tools built to support functional analysis of signaling networks. They enable to compute the functional route modulus for directed graphs that model activation, inhibition, and synergy among molecular components of signal transduction networks. The Python packages in this repository are dependent on *igraph* and *numpy*, so, if you have not already, install them by running the appropriate pip commands.

This notebook is meant to be a comprehensive overview of the main functionality of the packages in rtmod, how they work, and how to use them. We will review the methods of the **mfrpy** and **pmodpy** packages.

## Motivation
Conventional shortest-path algorithms are extremely refined and efficient. However, they are not necessarily applicable to the modeling of all types of networks. Cell signal transduction networks constitute one such example. In these instances, connections between subcomponents can be represented as directed edges, but there is no standardized representation for certain relationships between components, specifically those of conditional dependence (synergy) or inhibitory actions of some components on others. Taking these notions into account, our interest shifts from finding *minimal paths* from a source node to a target node to obtaining *minimal functional routes* by which a signal can travel from source to target, given these relationships between vertices in the network. One of the core aims of this project is to capture these important relationships graphically in order to better represent these prominent systems. In order to do this, we deal with *expanded graphs*, which are signal transduction networks that have been expanded (see below) to make computational algorithms more straightforward. We draw heavily from ideas developed by Reka Albert and colleagues at Pennsylvania State University and by Nathan Albin, Pietro Poggi-Corradini, and colleagues at Kansas State University, who have layed much of the theoretical groundwork for these concepts.

#### Synergy
A regular (activating) connection between two components of a signal transduction network can be represented by a directed edge between two nodes on the graph of the network. But how can we capture the fact that some components are only activated when two (or more) of its predecessors are activated in tandem? This is where the concept of synergy comes in. When the activation of a component is conditionally dependent on the simultaneous activation of several other components, the component's corresponding node on the graph is said to have a synergistic set of predeccessors consisting of precisely the nodes of the conditional components. Alternatively, we can say that the edges between these nodes have the same synergy value, which is the method by which this charecteristic is employed in the actual algorithm. 

#### Inhibition
It is often the case in cellular networks that one component inhibits the activation of another. This too, can be captured graphically. A certain node can be inhibited by one of its predeccessors, so instead of drawing an activating edge between them we might want to consider calling the edge between them inhibitory. More generally, we can assign every edge a sign +1 or -1 that distinguishes it as activatory or inhibitory.

### Theoretical Work
How do we represent attributes such as synergy and inhibition graphically? One way is by adding new conventions, such as dashed and barred edges, but this is largely unintuitive and can lead to ambiguity and confusion. Instead, we can transform the entire graph with the appropriate attributes into an "expanded" version of itself - a directed graph with only activating edges but with no ambiguity as to which components are synergistic and which are inhibitory. This is done by introducing new vertices: composite and inhibitory nodes. Composite nodes link a conditionally dependent node to its synergistic set of predecessors, while inhibitory nodes are activated when one component inhibits another. For example, if A inhibits B, we can also say that NOT A activates B AND A activates NOT B. By employing these transformations we can circumvent possible confusion and create programs which are capable of working with these notions.

### Empirical Networks
To illustrate the full functionality, we will implement the package on a model network, obtained from Juilee Thakar et al's 2007 publication on immune responses following *Bordetellae* bacteria invasions. We aim in particular to demonstrate numerical methods of graph expansion, minimal funcitonal route calculation, and modulus computation.

## mfrpy
**mfrpy** is a package for calculating the minimal functional routes of a given graph. 

#### igraph objects
The **igraph** package encodes graphs as operable Python objects. Edges and vertices are encoded in lists, making it possible to iteritavely attach attributes to each element based on certain criteria. To familiarize ourselves with the format, we can initialize an example graph:


In [4]:
from igraph import Graph, plot
g = Graph.Read_GraphML("bordetellaeGraph.xml")
g.vs["name"] = g.vs["id"]
g.vs["label"] = g.vs["name"]
plot(g, vertex_label_size = 8, vertex_size = (25), edge_arrow_size = 0.5, 
     vertex_color = "white", bbox=(0, 0, 700, 400))

AttributeError: Plotting not available; please install pycairo or cairocffi

Our test graph is a directed cyclic graph on 6 labeled vertices. Given prior knowledge of the network this graph represents, we can include synergy and inhibition quite easily by attaching the appropriate attributes to the edge sequence. Here, synergistic edges are color-coded.

In [6]:
i=1
indices = []
while i <= max(g.es["synergy"]):
    indices.append(
    [syn for syn,val in enumerate(g.es["synergy"]) if val==i]
    )
    i += 1
synergies = []
for ind in indices:
    syn = [g.get_edgelist()[i] for i in ind]
    synergies.append(syn)

for edge in g.get_edgelist():
    print(g.vs[edge[0]]["name"], "->", g.vs[edge[1]]["name"], 
    int(g.es[g.get_edgelist().index(edge)]["synergy"]), 
    bool(g.es[g.get_edgelist().index(edge)]["inhibition"]))

colors = ["#345995", "#70161E", "#BD4089", "#7EA16B", "#42213D", "#FB4D3D", 
          "#445552", "#294D4A", "#C33C54", "#254E70", "#8EE3EF"]

for group in synergies:
    for edge in group:
        g.es.select(_source_in=[edge[0]], 
                    _target_in=[edge[1]])["color"] = colors[synergies.index(group)]
        
plot(g, vertex_color = "white", vertex_label_size = 8, vertex_size = 25, edge_arrow_size = 0.5, bbox=(0, 0, 700, 400))

Bt -> EC 0 False
Bt -> Th2RC 0 True
Bt -> Th1RC 0 True
EC -> PIC 1 False
Th2RC -> PIC 1 True
PIC -> DC 0 False
DC -> T0 0 False
Th1RC -> Th2RC 2 True
DC -> Th2RC 2 False
T0 -> Th2RC 2 False
T0 -> Th1RC 3 False
PIC -> Th1RC 3 False
DC -> Th1RC 3 False
Th2RC -> Th2C 4 False
T0 -> Th2C 4 False
DC -> Th2C 4 False
PIC -> MP 0 False
Th1RC -> MP 0 False
Th2C -> BC 0 False
BC -> Cab 0 False
BC -> Oab 0 False
Bt -> AgAb 5 False
Cab -> AgAb 5 False
Bt -> AgAb 5 False
Oab -> AgAb 6 False
PIC -> RP 0 False
Bt -> Cp 0 False
Cab -> Cp 7 False
AgAb -> Cp 7 False
AgAb -> AP 8 False
RP -> AP 8 False
AgAb -> AP 8 False
MP -> AP 9 False
Cp -> AP 10 False
Cab -> AP 10 False
RP -> AP 8 False
Cab -> AP 10 False
Cp -> AP 10 False
MP -> AP 9 False
AP -> PH 0 False


AttributeError: Plotting not available; please install pycairo or cairocffi

### update_expand.py

This is a collection of methods for generating boolean update tables for the input graph, and converting these input tables into an expanded graph with composite nodes and logical NOT nodes to capture the behaviors of synergistic and inhibitory edges. We first obtain an update table with a zero-order logical expression for every node in the graph. The source node will always have an empty expressions since it lacks predecessors:

In [7]:
from mfrpy import update_expand
table = update_expand.updates(g, g.es["synergy"], g.es["inhibition"], 1)

Bt = 
EC = Bt
Th1RC = (T0&PIC&DC)|~Bt
Th2RC = (~Th1RC&DC&T0)|~Bt
PIC = (EC&~Th2RC)
DC = PIC
T0 = DC
Th2C = (Th2RC&T0&DC)
Cab = BC
Oab = BC
AgAb = (Bt&Cab)|(Bt&Oab)
Cp = (Cab&AgAb)|Bt
RP = PIC
AP = (AgAb&RP)|(AgAb&MP)|(Cp&Cab&RP)|(Cab&Cp&MP)
PH = AP
MP = Th1RC|PIC
BC = Th2C
~Th1RC = (Bt&~DC)|(Bt&~PIC)|(Bt&~T0)
~Th2RC = (Bt&Th1RC)|(Bt&~DC)|(Bt&~T0)
~PIC = Th2RC|~EC


We can now use this table to accurately expand our graph with the considerations of synergy and inhibition. Here, '~' is the logical NOT, whereas '&' and '|' are the logical AND and OR respectively. The drawback of doing this process graph-theoretically is the challenge of handling edges which have both attributes, as it is not always clear which expansion to perform first. By computing via the update table, there is no ambiguity and both attributes are expanded in tandem. We can now expand the test graph by running

In [8]:
expg = update_expand.expand(g, table, 0)

isolated = []
for node in expg.vs["name"]:
    if expg.neighbors(expg.vs["name"].index(node)) == []:
        isolated.append(expg.vs["name"].index(node))

expg.delete_vertices(isolated)

plot(expg, vertex_color = "white", vertex_label_size = 7, vertex_size = 26, edge_arrow_size = 0.5, bbox=(0, 0, 700, 400))

AttributeError: Plotting not available; please install pycairo or cairocffi

### sgmfr.py

This is a method for calculating the minimal functional routes of an expanded graph provided by `update_expand()`. The method uses a top-down subgraph-growing algorithm to generate minimal functional routes in the expanded graph. Starting from a target node, the algorithm examines its predecessors. If there are no composite nodes, it proceeds by adding predecessors one at a time until it reaches the source node, at which point it will pursue a different chain of predecessors, much like a normal shortest-path algorithm. However, if it encounters a composite node, the method appends all of the predecessors as one synergistic set in the minimal functional route. sgmfr can handle both cyclic and acyclic graphs, but to avoid redundancy that may occur from cyclic graphs, cycles are cleaved after all minimal functional routes are generated. It should be noted that inhibitory nodes are treated the same as any other node, as their nature is encoded in the expanded graph. To obtain the minimal functional routes, we execute

In [16]:
from mfrpy import sgmfr
g_mfrs = sgmfr.get_mfrs(g, 0, 14, 0, "es")

for mfr in g_mfrs[0]:
    print(g_mfrs[0].index(mfr)+1, ":", mfr)
    
for edge in g_mfrs[0][7]:
    expg.es.select(edge)["color"] = "red"
plot(expg, vertex_color = "white", vertex_label_size = 7, vertex_size = 26, edge_arrow_size = 0.5, bbox=(0, 0, 700, 400))

17
1 : [0, 1, 3, 4, 5, 6, 7, 13, 14, 15, 18, 19, 22, 23, 25, 31, 35, 39]
2 : [0, 1, 3, 4, 5, 6, 7, 13, 14, 15, 17, 18, 19, 22, 23, 31, 38, 39]
3 : [0, 1, 3, 4, 5, 6, 7, 13, 14, 15, 18, 19, 25, 26, 35, 36, 37, 39]
4 : [0, 1, 3, 4, 5, 6, 7, 13, 14, 15, 17, 18, 19, 26, 36, 37, 38, 39]
5 : [0, 1, 3, 4, 5, 6, 7, 13, 14, 15, 18, 20, 23, 24, 25, 31, 35, 39]
6 : [0, 1, 3, 4, 5, 6, 7, 13, 14, 15, 17, 18, 20, 23, 24, 31, 38, 39]
7 : [0, 1, 3, 4, 5, 6, 7, 13, 14, 15, 16, 18, 19, 22, 23, 31, 38, 39]
8 : [0, 1, 3, 4, 5, 6, 7, 13, 14, 15, 18, 19, 19, 19, 22, 23, 25, 27, 28, 35, 36, 37, 39]
9 : [0, 1, 3, 4, 5, 6, 7, 13, 14, 15, 17, 18, 19, 19, 19, 22, 23, 27, 28, 36, 37, 38, 39]
10 : [0, 1, 3, 4, 5, 6, 7, 13, 14, 15, 16, 18, 19, 26, 36, 37, 38, 39]
11 : [0, 1, 3, 4, 5, 6, 7, 13, 14, 15, 16, 18, 20, 23, 24, 31, 38, 39]
12 : [0, 1, 3, 4, 5, 6, 7, 13, 14, 15, 18, 19, 19, 20, 23, 24, 25, 27, 28, 35, 36, 37, 39]
13 : [0, 1, 3, 4, 5, 6, 7, 13, 14, 15, 16, 18, 19, 19, 19, 22, 23, 27, 28, 36, 37, 38, 39]
14 

AttributeError: Plotting not available; please install pycairo or cairocffi

The arguments of the **get_mfrs()** method are as follows:
1) the signal transduction igraph **graph** in question; expansion is handled internally so there is no need to expand it yourself beforehand

2) the integer **source node** from where you wish to compute minimal functional routes

3) the corresponding integral **target node**

4) Boolean **verbose** parameter; indicative of whether the method displays the results of its computations

5) String **format** of returned mfrs, of which there are three options:

    (i) "em": an edge matrix
    (ii) "el": an edge list, simply an alternate representation of the edge matrix
    (iii) "es": a sequence of edge ids as they appear in the graph

## pmodpy

**pmodpy** is a Python implementation of the graph modulus developed by Albin, Poggi-Corradini, and colleagues at Kansas State University. This is an analogue to mathematical graphs of the classical modulus defined on families of curves in a domain of the complex plane, and it is parameterized by the exponent $p$ of a distance norm.

An important theoretical contribution of the graph modulus is that its value on the family of walks between two nodes in a  graph $G$ unifies three common conceptions of distance on graphs: the maximum flow / minimum cut weight ($p = 1$), the resistance distance ($p = 2$), and the shortest path length ($p = \infty$). Its calculation also induces both an extremal density function on the edges of $G$ and an optimal probability mass function on the family of objects (e.g. walks) of interest.

These can be understood as measures of the importance of each edge or object to the task that defines the family, e.g. connecting two nodes or connecting all nodes of the graph. Because our interest is to understand signal transduction through the components of an intracellular signaling network, our objects of interest are functional routes.

Our implementation can compute the modulus for walks (whose minimal objects are paths), spanning subgraphs (whose minimal objects are spanning trees), and functional routes (for graphs with inhibition and/or synergy). Our first goal is to compare the insights gained and predictions made by signal transduction network analysis using minimal paths (a conventional approach) versus minimal functional routes.

### modsubfamily

The graph modulus on $G$ is defined for any (usually infinite) set of objects $\Gamma$ on $G$, which are operationalized as edge sets with multiplicity, $\Gamma \subset E(G) \times E(G)$).
The calculation of the modulus involves iteratively reweighting the edges of $G$ and obtaining a minimum object from $\Gamma$ under the new weights.
In the case of walks from $s$ to $t$ or spanning subgraphs, a minimum object can be obtained via an algorithm.
But what if such an algorithm is not available?

An _essential subfamily_ $\Gamma^\ast \subset \Gamma$ is finite while being sufficient to calculate the modulus.
We claim that the set of minimal objects in $\Gamma$—those that have no proper subobjects—constitute an essential subfamily when $G$ is finite.
In the case of functional routes, we have seen how to obtain these minimal objects, the MFRs.

The **modsubfamily** module provides two engines for calculating the graph modulus using an essential subfamily.
Each engine proceeds by optimizing one side of the modulus duality:
One optimizes the density function $\rho : E(G) \to \mathbb{R}$ on the edges of $G$ so as to minimize an energy function; as a result, we obtain not only the modulus but also the extremal density function.
Below we apply this engine to the _Bordetellae_ immune response network and our list of MFRs.

In [36]:
from pmodpy import modsubfamily
g_mfr_min = modsubfamily.get_minimum(g, g_mfrs[0])
g_mfr_mod_dens = modsubfamily.modulus_subfamily_density(g, g_mfrs[0])
print("Minimum functional route (uniform edge density):\n", g_mfr_min)
print("Functional route modulus (density optimization):\n", g_mfr_mod_dens[0])
print("Extremal density on the", g.ecount(), "edges of the graph:\n", g_mfr_mod_dens[1])

Minimum functional route (uniform edge density):
 [1 1 0 1 1 1 1 1 0 0 0 0 0 1 1 1 0 0 1 1 0 0 1 1 0 1 0 0 0 0 0 1 0 0 0 1 0
 0 0 1]
Functional route modulus (density optimization):
 0.06770395360768308
Extremal density on the 40 edges of the graph:
 [6.77039511e-02 6.77039511e-02 6.69694581e-06 6.77039511e-02
 6.77039511e-02 6.77039511e-02 6.77039511e-02 6.77039511e-02
 6.69694581e-06 6.69694581e-06 6.69694581e-06 6.69694581e-06
 6.69694581e-06 6.77039511e-02 6.77039511e-02 6.77039511e-02
 1.93439924e-02 1.93439955e-02 6.77039511e-02 4.12111018e-02
 2.64928572e-02 6.69694581e-06 1.17746113e-02 3.82674531e-02
 2.64928572e-02 2.90159844e-02 2.94365055e-02 6.69694581e-06
 6.69694581e-06 6.69694581e-06 6.69694581e-06 3.82674531e-02
 6.69694581e-06 6.69694581e-06 6.69694581e-06 2.90159844e-02
 2.94365055e-02 2.94365055e-02 3.86879742e-02 6.77039511e-02]


The other engine optimizes a probability mass function $\mu : \Gamma \to \mathbb{R}_{\geq 0}$, which only takes positive values on an essential subset $\Gamma^\ast$.
Below we apply this engine to the signal transduction network:

In [35]:
from pmodpy import modsubfamily
g_mfr_min = modsubfamily.get_minimum(g, g_mfrs[0])
g_mfr_mod_mass = modsubfamily.modulus_subfamily_mass(g, g_mfrs[0])
print("Functional route modulus (p.m.f. optimization):\n", g_mfr_mod_mass[0])
print("Optimal probability mass function on the", len(g_mfrs[0]), "MFRs:\n", g_mfr_mod_mass[1])

Functional route modulus (p.m.f. optimization):
 0.0677039517615075
Optimal probability mass function on the 15 MFRs:
 [6.30416321e-02 5.54416038e-02 1.95497603e-01 1.19641013e-01
 1.70023578e-01 1.10635860e-01 5.54416038e-02 4.81314638e-08
 4.95789273e-08 1.19641013e-01 1.10635860e-01 2.95690681e-08
 4.95789273e-08 2.85204518e-08 2.85204518e-08]


Up to our default tolerance, the modulus calculations agree.
We also now have theoretically-motivated importance rankings both on the edges of $G$ and on the minimal functional routes from bacteria (Bt) to phagocytosis (PH).
In the case $p = 2$ used here, the extremal density and optimal p.m.f. also have a probabilistic interpretation.

# References

Albin, N., Poggi-Corradini, P. Minimal subfamilies and the probabilistic interpretation for modulus on graphs. J Anal 24, 183–208 (2016). https://doi.org/10.1007/s41478-016-0002-9

Albin, N., Poggi-Corradini, P., Darabi Sahneh, F., and Goering, M. Modulus of families of walks on graphs (2014). http://adsabs.harvard.edu/abs/2014arXiv1401.7640A

Wang, R.-S., Sun, Z. and Albert, R. Minimal functional routes in directed graphs with dependent edges. Intl. Trans. in Op. Res., 20: 391-409 (2013). https://doi.org/10.1111/itor.12007

Wang, R.-S., Albert, R. Elementary signaling modes predict the essentiality of signal transduction network components. BMC Syst Biol 5, 44 (2011). https://doi.org/10.1186/1752-0509-5-44

Thakar J., Pilione M., Kirimanjeswara G., Harvill E.T., Albert R. Modeling Systems-Level Regulation of Host Immune Responses. PLoS Comput Biol 3, 6 (2007). https://doi.org/10.1371/journal.pcbi.0030109